In [4]:
import pandas as pd

# -----------------------------------
# 1) CSV 불러오기
# -----------------------------------
csv_path = "../data_csv/11_10/춘천시_버스정류장노선정보_20250212.csv"
df = pd.read_csv(csv_path, encoding="cp949")



In [ ]:
-----------------------------
# 2) SQL 파일 생성
# -----------------------------------
table_name = "chuncheon_bus_stop_routes"
sql_path = "chuncheon_bus_routes.sql"

# CREATE TABLE 문
create_table_sql = f"""
DROP TABLE IF EXISTS {table_name};

CREATE TABLE {table_name} (
    route_no    INT,
    route_code  DOUBLE,
    stop_seq    DOUBLE,
    stop_id     BIGINT,
    stop_name   VARCHAR(255),
    lon         DOUBLE,
    lat         DOUBLE,
    data_date   DATE
);
"""

# INSERT 문 생성
insert_sql = ""
for _, row in df.iterrows():
    insert_sql += (
        f"INSERT INTO {table_name} "
        f"(route_no, route_code, stop_seq, stop_id, stop_name, lon, lat, data_date) "
        f"VALUES ("
        f"{int(row['노선번호'])}, "
        f"{float(row['노선'])}, "
        f"{float(row['정류장순서'])}, "
        f"{int(row['정류장'])}, "
        f"'{row['정류장명']}', "
        
        f"{float(row['경도'])}, "
        f"{float(row['위도'])}, "
        f"'{row['데이터기준일']}');\n"
    )

# 파일로 저장
with open(sql_path, "w", encoding="utf-8") as f:
    f.write(create_table_sql)
    f.write("\n")
    f.write(insert_sql)

print("SQL 파일 생성 완료:", sql_path)

ValueError: invalid literal for int() with base 10: '2-S'

In [10]:
import pandas as pd
import math

# --- 설정 ---
# 읽을 CSV 파일 이름
csv_file_name = "../data_csv/11_11/강원도업종.csv"

# 생성할 SQL 파일 이름
sql_file_name = "insert_kangwondo_businesses.sql"

# 생성할 테이블 이름 (따옴표 없음)
table_name = "kangwondo_businesses"
# ----------------

def clean_value_mysql(value):
    """
    Python 데이터를 SQL 문자열로 안전하게 변환합니다.
    - None 또는 NaN -> 'NULL'
    - 문자열 -> 작은따옴표로 감싸고, 내부의 작은따옴표 이스케이프
    """
    if pd.isna(value):
        return "NULL"
    elif isinstance(value, str):
        # MySQL은 '를 ''로, \를 \\로 이스케이프합니다.
        cleaned_value = value.replace("\\", "\\\\").replace("'", "''")
        return f"'{cleaned_value}'"
    else:
        # 숫자나 다른 타입은 문자열로 변환
        return str(value)

# 1. CSV 파일 읽기 (한글 인코딩 처리)
try:
    df = pd.read_csv(csv_file_name, encoding='cp949')
except UnicodeDecodeError:
    df = pd.read_csv(csv_file_name, encoding='utf-8-sig')
except Exception as e:
    print(f"파일 읽기 오류: {e}")
    df = pd.DataFrame() # 오류 시 빈 데이터프레임

if not df.empty:
    print(f"'{csv_file_name}' 파일 읽기 성공. {len(df)}개 행 발견.")
    
    # 2. CREATE TABLE 쿼리 생성
    # 컬럼명 리스트 (따옴표 없음)
    columns = [col for col in df.columns]
    column_sql = ", ".join(columns)
    
    create_table_sql = f"""
-- 강원도 업종 테이블 생성 (MySQL용, 따옴표 없음)
DROP TABLE IF EXISTS {table_name};
CREATE TABLE {table_name} (
    업종 VARCHAR(255) CHARACTER SET utf8mb4,
    업소명 VARCHAR(255) CHARACTER SET utf8mb4,
    주소 VARCHAR(255) CHARACTER SET utf8mb4,
    연락처 VARCHAR(50)
);
\n"""

    # 3. SQL 파일 쓰기 시작
    try:
        with open(sql_file_name, 'w', encoding='utf-8') as f:
            # 파일 상단에 데이터베이스 선택 주석 추가
            f.write("-- 사용할 데이터베이스를 선택하세요 (예: USE your_database_name;)\n")
            
            # CREATE TABLE 문 쓰기
            f.write(create_table_sql)
            f.write("\n")
            
            values_batch = []
            
            # INSERT 문 시작 부분 (따옴표 없음)
            insert_prefix = f"INSERT INTO {table_name} ({column_sql}) VALUES \n"
            
            # 4. DataFrame을 행별로 순회하며 VALUES 생성
            for row in df.itertuples(index=False):
                cleaned_row = [
                    clean_value_mysql(getattr(row, '업종')),
                    clean_value_mysql(getattr(row, '업소명')),
                    clean_value_mysql(getattr(row, '주소')),
                    clean_value_mysql(getattr(row, '연락처'))
                ]
                values_batch.append(f"({', '.join(cleaned_row)})")
            
            # 5. INSERT 쿼리 파일에 쓰기
            if values_batch:
                # 모든 데이터를 하나의 INSERT 문으로 합쳐서 씀
                f.write(insert_prefix + ",\n".join(values_batch) + ";\n\n")
                
        print(f"'{sql_file_name}' 파일 생성이 완료되었습니다.")
        print(f"총 {len(df)}개의 행이 파일에 저장되었습니다.")

    except Exception as e:
        print(f"SQL 파일 생성 중 오류: {e}")

'../data_csv/11_11/강원도업종.csv' 파일 읽기 성공. 356개 행 발견.
'insert_kangwondo_businesses.sql' 파일 생성이 완료되었습니다.
총 356개의 행이 파일에 저장되었습니다.
